In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
!unzip /gdrive/MyDrive/Proyecto_Probabilidad/probabilidad/cultivos_nacional.zip
!unzip /gdrive/MyDrive/Proyecto_Probabilidad/probabilidad/cultivos_nacional_filtrado.zip

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(dplyr)
library(tidyverse)
library(ggplot2)
install.packages("rmarkdown")
# install.packages("ggpubr")
# library(ggpubr)

# Se importan los archivos

In [ ]:
%%R
cultivos_code2name <- read.table("/gdrive/MyDrive/Proyecto_Probabilidad/probabilidad/equivalencias_cultivos_code.csv", sep=",", header=TRUE)
dep_names = read.table("/gdrive/MyDrive/Proyecto_Probabilidad/probabilidad/codigo_depto_a_nombre.csv",sep=",", header=TRUE)
regre <- read.table("/gdrive/MyDrive/Proyecto_Probabilidad/probabilidad/cultivos_nacional_filtrado.csv", sep=",", header=TRUE)

# Codigo para encontrar el departamento mas afectado

In [ ]:
%%R
df_dep_names = as.data.frame(dep_names)
colnames(df_dep_names) = c("departamento","nombre_departamento")

#Total del area sembrada desde 1920 hasta 2013 de cada departamento
suma_areas_sembradas = regre %>%
group_by(P_DEPTO) %>%
summarise(total_cultivado=sum(AREA_SEMBRADA))
df_suma_areas_sembradas = as.data.frame(suma_areas_sembradas)

df_suma_areas_sembradas = arrange(df_suma_areas_sembradas,desc(total_cultivado))
colnames(df_suma_areas_sembradas) <- c("departamento", "total_cultivos")
df_suma_areas_sembradas = na.omit(df_suma_areas_sembradas)
df_merge_suma_areas_sembradas = filter(df_suma_areas_sembradas, departamento != 63)
df_merge_suma_areas_sembradas = merge(df_merge_suma_areas_sembradas,df_dep_names,by="departamento")
print(df_merge_suma_areas_sembradas[,-c(1)])
#---------------------------------------------------------------------

# Total cultivos afectados
afectados = filter(regre,P_S6P60 != 12) #Elimina de la tabla los cultivos que no fueron afectados

# Calculo de el area perdida 
# (le restamos al area sembrada el area cosechada, para encontrar cuanto se perdio en cada reporte)
diferencia_areas <- afectados %>%
summarise(afectacion = P_S6P60,departamento = P_DEPTO,res_area = AREA_SEMBRADA-AREA_COSECHADA)
# print(diferencia_areas[1:10,])

# Total de area perdida (desde 1920 hasta 2013) de departamento 
# (Sumamos todas las perdidas de la anterior tabla para encontrar el total de area perdida)
suma_diferencia_areas = diferencia_areas %>%
group_by(departamento) %>%
summarise(total_afectado=sum(res_area))

df_suma_diferencia_areas = as.data.frame(suma_diferencia_areas)
df_suma_diferencia_areas = arrange(df_suma_diferencia_areas,desc(total_afectado))
df_suma_diferencia_areas = na.omit(df_suma_diferencia_areas)
df_suma_diferencia_areas = na.omit(full_join(df_suma_diferencia_areas,df_dep_names,by="departamento"))
print(df_suma_diferencia_areas[,-c(1)])

# Union de tablas
u_tablas = merge(df_merge_suma_areas_sembradas, df_suma_diferencia_areas, by = "departamento")
porcentajes_departamentos = u_tablas %>%
summarise(departamento,porcentaje_afectado=round((total_afectado*100)/total_cultivos, digits=5))
porcentajes_departamentos = merge(porcentajes_departamentos,df_dep_names[-c(19),],by="departamento")
porcentajes_departamentos = arrange(porcentajes_departamentos,desc(porcentaje_afectado))
print(porcentajes_departamentos[,-c(1)])

# pdf(file="area_afectada_nacional.pdf", width=9, height=8)
# ggplot(df_suma_diferencia_areas, aes(x=nombre_departamento, y=total_afectado, fill=total_afectado))+geom_bar(stat="identity")+ coord_flip()+labs(colors="Area afectada\n",x = "Departamento", y = "Area afectada", title = "Area de los cultivos de los departamentos afectados")

# Funciones para encontrar los cultivos afectados por departamento

In [ ]:
%%R
cultivos_sembrados <- function(df){
  suma_areas_sembradas = df %>%
  group_by(P_S6P46) %>%
  summarise(total_cultivado=sum(AREA_SEMBRADA))

  df_suma_areas_sembradas = as.data.frame(suma_areas_sembradas)
  df_suma_areas_sembradas = arrange(df_suma_areas_sembradas,desc(total_cultivado))
  colnames(df_suma_areas_sembradas) <- c("cultivo_id", "total_cultivos")
  df_suma_areas_sembradas = na.omit(df_suma_areas_sembradas)
  df_merge_suma_areas_sembradas = filter(df_suma_areas_sembradas)
  # print(df_merge_suma_areas_sembradas[1:10,]) #------------------
  #---------------------------------------------------------------------
  graf_afectadas = full_join(df_merge_suma_areas_sembradas, cultivos_code2name, by="cultivo_id")
  graf_afectadas = na.omit(graf_afectadas)
}

afec_por_cultivo <- function(afec) {
  # Calculo de el area perdida
  # (le restamos al area sembrada el area cosechada, para encontrar cuanto se perdio en cada reporte)
  diferencia_areas <- afec %>%
  summarise(afectacion = P_S6P60,cultivo_id = P_S6P46,res_area = AREA_SEMBRADA-AREA_COSECHADA)
  # print(diferencia_areas[1:10,])

  # Total de area perdida (desde 1920 hasta 2013) de cultivo_id
  # (Sumamos todas las perdidas de la anterior tabla para encontrar el total de area perdida)
  suma_diferencia_areas = diferencia_areas %>%
  group_by(cultivo_id) %>%
  summarise(total_afectado=sum(res_area))

  df_suma_diferencia_areas = as.data.frame(suma_diferencia_areas)
  df_suma_diferencia_areas = arrange(df_suma_diferencia_areas,desc(total_afectado))
  df_suma_diferencia_areas = na.omit(df_suma_diferencia_areas)
  # print(df_suma_diferencia_areas[1:10,]) #------------------
  graf_afectadas = full_join(df_suma_diferencia_areas, cultivos_code2name, by="cultivo_id")
  graf_afectadas = na.omit(graf_afectadas)

  # Union de tablas
  # u_tablas = merge(df_merge_suma_areas_sembradas, df_suma_diferencia_areas, by = "departamento")
  # porcentajes_departamentos = u_tablas %>%
  # summarise(departamento,porcentaje_afectado=round((total_afectado*100)/total_cultivos, digits=5))
  # porcentajes_departamentos = merge(porcentajes_departamentos,df_dep_names[-c(19),],by="departamento")
  # porcentajes_departamentos = arrange(porcentajes_departamentos,desc(porcentaje_afectado))
  # print(porcentajes_departamentos[,-c(1)])
}


# Codigo para encontrar el total afectado nacional por area

In [ ]:
%%R
cat("total nacional\n \n")
total_semb_nacional <- cultivos_sembrados(regre)
print(total_semb_nacional[1:10,])

cat("\n total afectado nacional \n \n")
afectados = filter(regre,P_S6P60 != 12)
# print(afectados[1:10,])
nacional_afec <- afec_por_cultivo(afectados)
print(nacional_afec[1:10,])
pdf(file="afectado_cultivo_nacional.pdf", width=9, height=8)
ggplot(nacional_afec[1:10,], aes(x=nombre, y=total_afectado, fill=total_afectado))+geom_bar(stat="identity")+ coord_flip()+labs(colors="Area afectada\n",x = "Cultivo", y = "Area afectada", title = "Top 10 cultivos mas afectados por area nacional")

# Cultivos afectados en Archipielago

In [60]:
%%R
df_afectados <- as.data.frame(afectados)
archi <- filter(regre, P_DEPTO == 88)

afectado_archi <- afec_por_cultivo(archi)
print(afectado_archi[1:10,])

# pdf(file="porcentaje_afectado_cultivo_archipielago.pdf", width=9, height=8)
# ggplot(afectado_archi[1:10,], aes(x=nombre, y=total_afectado, fill=total_afectado))+geom_bar(stat="identity")+ coord_flip()+labs(colors="Porcentaje afectado\n",x = "Cultivo", y = "Porcentaje afectado", title = "Top 10 cultivos mas afectados por area Archipielago")

   cultivo_id total_afectado     nombre
1   159201001      29.418644      Yuca 
2   131201001      11.271970    Banano 
3   123502001      10.615705   Ahuyama 
4   131801001       7.851949       Piña
5   159301001       7.008000      Ñame 
6   191999066       7.000000   Guandul 
7   191999056       5.490000     Fríjol
8   123102001       3.285500 Pimentón  
9   132301002       2.924528    Naranja
10  146001001       2.870230       Coco


# Cultivos afectados en Choco

In [62]:
%%R
choco <- filter(regre, P_DEPTO == 27)

afectado_choco <- afec_por_cultivo(choco)
print(afectado_choco[1:10,])


   cultivo_id total_afectado         nombre
1   159201001      3674.9850          Yuca 
2   112201002      1477.4634 Maíz Amarillo 
3   159301001      1067.0796          Ñame 
4   131301001       513.6640       Platano 
5   112201001       499.9409   Maíz Blanco 
6   131906001       496.1912        Borojo 
7   129001001       350.8791       Guatila 
8   131201001       314.3116        Banano 
9   146001001       247.9906           Coco
10  180201002       139.8706  Caña panelera


In [63]:
%%R
graf_barra <- function(df,eje_x,eje_y){
    ggplot(data=df, aes(x=eje_x, y=eje_y, fill=eje_x)) + 
    geom_bar(stat="identity", position="dodge")
}
graf_puntos <- function(df,eje_x,eje_y, ex_color){
    ggplot(data=df, aes(x=eje_x, y=eje_y, color=ex_color)) + 
    geom_point(size = 5, alpha = 1/2) + scale_color_gradientn(colours = rainbow(12))
}

In [ ]:
%%R
# print(archi[1:10,])
afec_archi_graf = filter(archi, P_S6P46==159201001)
print(afec_archi_graf[1:10,])

graf_puntos(afec_archi_graf,afec_archi_graf$P_S6P59_UNIF, afec_archi_graf$AREA_COSECHADA, afec_archi_graf$P_S6P60)+labs(x="x", y="y", color="afec\n", title="Regre")

In [ ]:
%%R
# Total cultivos afectados
afectados = filter(regre,P_S6P60 != 12) #Elimina de la tabla los cultivos que no fueron afectados

cultivo_nacional_afectado <- afec_por_cultivo(afectados)

In [ ]:
%%R
graf_puntos(afectados, afectados$P_S6P59_UNIF,afectados$AREA_SEMBRADA, afectados$P_S6P60)+labs(x="x", y="y", color="afec\n", title="Regre")

# Regrecion lineal con el departamento Choco

In [72]:
%%R
regre_lin <- function(ex_data,ex_x,ex_y, ex_color){
    lm.fit <- lm(data= ex_data,ex_y ~ ex_x,na.action=na.exclude)
    sumar = summary(lm.fit)
    cat("Regresion lineal \n \n")
    print(summary(lm.fit))
    cat("\n")
}
choco <- as.data.frame(afectados[afectados$P_DEPTO==27,])
choco <- na.omit(choco)
regre_lin(choco, choco$P_S6P47B, choco$AREA_COSECHADA, choco$P_S6P47B)

Regresion lineal 
 

Call:
lm(formula = ex_y ~ ex_x, data = ex_data, na.action = na.exclude)

Residuals:
    Min      1Q  Median      3Q     Max 
 -1.425  -0.830  -0.473   0.298 170.993 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -57.358814   2.906102  -19.74   <2e-16 ***
ex_x          0.029206   0.001451   20.12   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.218 on 26601 degrees of freedom
Multiple R-squared:  0.015,	Adjusted R-squared:  0.01496 
F-statistic:   405 on 1 and 26601 DF,  p-value: < 2.2e-16




# Regrecion exponencial con el departamento Choco

In [73]:
%%R
regre_exp <- function(ex_data,ex_x,ex_y, ex_color){
    lm.fit <- lm(data= ex_data,log(ex_y) ~ ex_x)
    cat("Regresion log \n \n")
    print(summary(lm.fit))
    cat("\n")
}
choco <- as.data.frame(afectados[afectados$P_DEPTO==27,])
choco <- na.omit(choco)
regre_exp(choco, choco$P_S6P47B, choco$AREA_COSECHADA, choco$P_S6P47B)

Regresion log 
 

Call:
lm(formula = log(ex_y) ~ ex_x, data = ex_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.5920 -0.9594  0.2948  1.1500  6.3331 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -80.173674   2.025471  -39.58   <2e-16 ***
ex_x          0.039619   0.001012   39.17   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.546 on 26601 degrees of freedom
Multiple R-squared:  0.05453,	Adjusted R-squared:  0.05449 
F-statistic:  1534 on 1 and 26601 DF,  p-value: < 2.2e-16




# Regrecion polinomial con el departamento Choco

In [74]:
%%R
regre_pol <- function(ex_data,ex_x,ex_y, ex_color){
    lm.fit <- lm(data= ex_data,log(ex_y) ~ poly(ex_x,2))
    cat("Regresion log \n \n")
    print(summary(lm.fit))
    cat("\n")
    # ggplot(ex_data, aes(x=ex_x, y=ex_y,color=ex_color))+geom_smooth(method="exp", colour = "red")+ 
    # geom_point(size = 5, alpha = 1/5) + scale_color_gradientn(colours = rainbow(6))
}
choco <- as.data.frame(afectados[afectados$P_DEPTO==27,])
choco <- na.omit(choco)
regre_pol(choco, choco$P_S6P47B, choco$AREA_COSECHADA, choco$P_S6P47B)

Regresion log 
 

Call:
lm(formula = log(ex_y) ~ poly(ex_x, 2), data = ex_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.6435 -0.9682  0.3064  1.1011  6.3602 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -0.840449   0.009454  -88.90   <2e-16 ***
poly(ex_x, 2)1 60.562662   1.541945   39.28   <2e-16 ***
poly(ex_x, 2)2 18.801724   1.541945   12.19   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.542 on 26600 degrees of freedom
Multiple R-squared:  0.05978,	Adjusted R-squared:  0.05971 
F-statistic: 845.7 on 2 and 26600 DF,  p-value: < 2.2e-16




# Convertir Jupyter a Rmarkdown

In [58]:
%%R
# library(rmarkdown)
ipynb_file = rmarkdown:::convert_ipynb("/content/proyecto_proba.ipynb")
st_nb_rmd = xfun::file_string(ipynb_file)
fileConn <- file("/content/proyecto_proba.rmd")
writeLines(st_nb_rmd, fileConn)
close(fileConn)